# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import re
import numpy as np
import math

In [2]:
df = pd.read_csv('main_task.xls')

In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [4]:
# Также создадим колонку является ли город столицей или нет

def capital(City):
    
    capital = ['Paris', 'Stockholm', 'London', 'Berlin',
           'Bratislava', 'Vienna', 'Rome', 'Madrid',
           'Dublin', 'Brussels', 'Warsaw', 'Budapest',
           'Copenhagen', 'Amsterdam', 'Lisbon', 'Prague',
           'Oslo', 'Helsinki', 'Edinburgh', 'Ljubljana',
           'Athens', 'Luxembourg']
    
    if City in capital:
        return 1
    else:
        return 0


df['Capital'] = df['City'].apply(capital)

In [5]:
# Сразу же создадим dummy признак по городоам
df = pd.concat([df.drop('City', axis=1), pd.get_dummies(df['City'])], axis=1)

# И заменим NaN значение в признаке Number of Reviews
df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

In [6]:
#Заменим также значения в колокне Price Range

def price(x):
    if x == '$':
        return 1
    elif x == '$$ - $$$':
        return 2
    elif x == '$$$$':
        return 3

df['Price Range'] = df['Price Range'].apply(price)
df['Price Range'] = df['Price Range'].fillna(2)

In [7]:
# Напимшем функцию по обработке ячеек, где есть не одно значение

def splitting_row(str_val):
    if pd.isna(str_val):
        return ["Unknown"]
    str_val = str_val.strip('[]')
    str_val = str_val.replace("\'",'')
    str_val = str_val.split(",")
    return str_val

def counter(row):
    for i in row:
        if i=='Unknown':
            return 1
        else:
            return len(i)

In [8]:
# Применим функцию

df['Cuisine Style'] = df['Cuisine Style'].apply(splitting_row)

# И сразу же приведем колонку в порядок, разбив значения и приведя к общему виду

df = df.explode('Cuisine Style')
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x.lower().lstrip())

#df['Cuisine Style']= df['Cuisine Style'].apply(counter)

In [9]:
# И также создадим dummy признак по кухням

df = pd.concat([df.drop('Cuisine Style', axis=1), pd.get_dummies(df['Cuisine Style'])], axis=1)

In [10]:
# Теперь поработаем над датами через паттерн

pattern = re.compile('\d+\/\d+\/\d+')

df['Reviews'] = df['Reviews'].apply(pattern.findall)

df['Reviews'] =df['Reviews'].apply(lambda x:[x[-2], x[-1]] if len(x)>=3
                                  else x)

# Так как для модели нам нужны исключительно числа, то создадим колонку с разницей оставленных отзывов в днях, переведем их
# в формат integer

df['Reviews1'] = df['Reviews'].apply(lambda x: x[0] if len(x)>=1 else None)
df['Reviews2'] = df['Reviews'].apply(lambda x: x[1] if len(x)==2 else None)

df['Reviews1'] = pd.to_datetime(df['Reviews1'])
df['Reviews2'] = pd.to_datetime(df['Reviews2'])
df['Reviews_Dif'] = df['Reviews1'] - df['Reviews2']
df['Reviews_Dif'] = pd.to_numeric(df['Reviews_Dif'].dt.days, downcast='integer')

In [11]:
# Заменим NaN значения на среднее

df['Reviews_Dif'] = df['Reviews_Dif'].fillna(df['Reviews_Dif'].mean())

In [12]:
# Удалим ненформативнык колонки, в том числе и "Price Range"

df = df.drop(['Reviews', 
              'URL_TA', 'ID_TA', 'Reviews1',
             'Reviews2'], axis=1)

In [13]:
df.head()

,Restaurant_id,Ranking,Rating,Price Range,Number of Reviews,Capital,Amsterdam,Athens,Barcelona,Berlin,...,uzbek,vegan options,vegetarian friendly,venezuelan,vietnamese,welsh,wine bar,xinjiang,yunnan,Reviews_Dif
0,id_5569,5570.0,3.5,2.0,194.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41.0
0,id_5569,5570.0,3.5,2.0,194.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41.0
0,id_5569,5570.0,3.5,2.0,194.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41.0
1,id_1535,1537.0,4.0,2.0,10.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,382.0
2,id_352,353.0,4.5,3.0,688.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104896 entries, 0 to 39999
Columns: 164 entries, Restaurant_id to Reviews_Dif
dtypes: float64(5), int64(1), object(1), uint8(157)
memory usage: 22.1+ MB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [15]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [16]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [17]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104896 entries, 0 to 39999
Columns: 162 entries, Ranking to Reviews_Dif
dtypes: float64(4), int64(1), uint8(157)
memory usage: 20.5 MB


# Создаём, обучаем и тестируем модель

In [19]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [20]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [21]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.09769600366076871
